# **Grab Hydromet Data**

By: Bridget Bittmann

Date created: 05/24/2022

Date modified: 08/11/2022

Purpose: This script uses a pre-existing csv of hydromet data and takes summary statistics on Lucky Peak inflows and maximum reservoir storage for the three reservoirs in the system based on the water year. 


In [ ]:
## --------------- ## 
## IMPORT PACKAGES ## 
## --------------- ## 

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
#Navigate to your directory

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Navigate to workspace

%cd /content/drive/MyDrive/spatial_colab/datasets/
%ls

/content/drive/MyDrive/spatial_colab/datasets
climate_stats/         irrigation_companies/  lcmap_files/   subset_test_shp/
diversion_timeseries/  irrig_lbrb/            masked/
extra_figures/         IrrMapper/             output_files/
hydromet_data/         LBRB_shp/              POUs/


In [ ]:
## --------------------- ## 
## IMPORT HDYROMET FILES ##
## --------------------- ## 

hydromet = pd.read_csv('hydromet_data/reservoir_data.csv', parse_dates=['DateTime'])
hydromet = hydromet[hydromet['DateTime']>='01-01-1986'].drop('Unnamed: 0', axis=1)

In [ ]:
## ---------------------------------------------- ## 
## SUM INFLOWS AND RESERVOIR FILL FOR WATER YEARS ## 
## ---------------------------------------------- ## 

years = np.arange(1987, 2021)

hydromet = hydromet[hydromet['wy'].isin(years)]

LP_inflows = pd.DataFrame(hydromet.groupby('wy').luc_id.sum())
max_fill = hydromet.groupby('wy').total_storage.max()

carryover = []
for i in years:
  subset = hydromet[hydromet['wy'] == i]
  date = str(i-1)+'-10-31'
  carry = int(subset['total_storage'][subset['DateTime'] == date].values)
  carryover.append([carry, i])

carryover = pd.DataFrame(carryover, columns=['Carryover','Year'])

hydromet_new = LP_inflows.merge(max_fill, on='wy', how='left').reset_index()
hydromet_new = hydromet_new.rename({'luc_id':'LP_inflows', 
                                    'total_storage':'Max_Fill',
                                    'wy':'Year'}, axis=1)
hydromet_new = hydromet_new.merge(carryover, on='Year', how='left')

hydromet_new.to_csv('hydromet_data/mode_input_hydromet.csv')